In [1]:
import pandas as pd 
import re

In [2]:
df = pd.read_csv("data/dataset_B_05_2020.csv")
url_df = df[['url', "status"]]

In [3]:
url_df.head()

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


In [ ]:
df.columns

In [ ]:
import requests
from bs4 import BeautifulSoup

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
except Exception as e:
    response = ""
    soup = -999

In [ ]:
# length_url
url_df['url'].apply(lambda x: len(x))

# length_hostname
url_df['url'].apply(lambda x: len(x.split("/")[2]))

# nb_dot
url_df['url'].apply(lambda x: x.count("."))

# nb_hyphens
url_df['url'].apply(lambda x: x.count("-"))

# nb_at
url_df['url'].apply(lambda x: x.count("@"))

# nb_qm
url_df['url'].apply(lambda x: x.count("?"))

# nb_and
url_df['url'].apply(lambda x: x.count("&"))

# nb_eq
url_df['url'].apply(lambda x: x.count("="))

# nb_underscore
url_df['url'].apply(lambda x: x.count("-"))

# nb_tilde
url_df['url'].apply(lambda x: x.count("~"))

# nb_percentage
url_df['url'].apply(lambda x: x.count("%"))

# nb_slash
url_df['url'].apply(lambda x: x.count("/"))

# nb_star
# Note: We only have 8 url with stars in our data set
url_df['url'].apply(lambda x: x.count("*"))

# nb_colon
url_df['url'].apply(lambda x: x.count(":"))

# nb_comma
url_df['url'].apply(lambda x: x.count(","))

# nb_semicolumn
url_df['url'].apply(lambda x: x.count(";"))

# nb_doller
url_df['url'].apply(lambda x: x.count("$"))

# nb_space
# Note: Result is not matching
url_df['url'].apply(lambda x: x.count(" "))

# nb_www
url_df['url'].apply(lambda x: x.count("www"))

# nb_com
url_df['url'].apply(lambda x: x.count("com"))

# nb_dslash
# Note: Result is not matching
url_df['url'].apply(lambda x: x.count("//"))

# https token
url_df['url'].apply(lambda x: 0 if "https" in x else 1)

# http_in_path 
# Note some percentage of values are not matching (but maximum are)
url_df['url'].apply(lambda url: ("".join(url.split('/')[1:])).count("http"))


# shortening_service
def shortening_service(full_url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      full_url)
    if match:
        return 1
    else:
        return 0

In [ ]:
import whois
from datetime import datetime

def get_domain_age(url):
    try:
        # Get the whois information for the domain
        domain_info = whois.whois(url)

        # Extract the creation date from the whois information
        creation_date = domain_info.creation_date

        # Check if the creation date is a list and get the first element
        if isinstance(creation_date, list):
            creation_date = creation_date[0]

        # Calculate the age of the domain in years
        if creation_date:
            today = datetime.now()
            age = today.year - creation_date.year - ((today.month, today.day) < (creation_date.month, creation_date.day))
            return age
        else:
            return "Creation date not available"

    except whois.parser.PywhoisError as e:
        return f"Error: {e}"

# Example usage
url = "http://www.crestonwood.com/router.php	"
domain_age = get_domain_age(url)

print(f"The age of the domain {url} is: {domain_age}")


In [ ]:
import requests
from bs4 import BeautifulSoup

def is_indexed_by_google(url):
    try:
        # Send a GET request to Google with the site: operator
        response = requests.get(f'https://www.google.com/search?q=site:{url}')

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Check if the search results contain the website URL
            search_results = soup.find_all('a', href=True)
            for result in search_results:
                if url.lower() in result['href'].lower():
                    return 1
            return 0
        else:
            return 0  # Return 0 for failure to retrieve data

    except Exception as e:
        return 0  # Return 0 for errors

# Example usage
website_url = "example.com"
result = is_indexed_by_google(website_url)

print(f"The website {website_url} is{' ' if result else ' not '}indexed by Google.")


In [ ]:
import dns.resolver

def get_dns_records(domain):
    try:
        # Use the default resolver
        resolver = dns.resolver.Resolver()

        # Get all DNS records for the domain
        response = resolver.resolve(domain, 'A')  # 'A' record for IPv4 addresses
        a_records = [record.address for record in response]

        response = resolver.resolve(domain, 'AAAA')  # 'AAAA' record for IPv6 addresses
        aaaa_records = [record.address for record in response]

        response = resolver.resolve(domain, 'MX')  # 'MX' record for mail servers
        mx_records = [(record.preference, str(record.exchange)) for record in response]

        response = resolver.resolve(domain, 'NS')  # 'NS' record for name servers
        ns_records = [str(record.target) for record in response]

        response = resolver.resolve(domain, 'CNAME')  # 'CNAME' record for aliases
        cname_records = [str(record.target) for record in response]

        return {
            'A': a_records,
            'AAAA': aaaa_records,
            'MX': mx_records,
            'NS': ns_records,
            'CNAME': cname_records
        }

    except dns.resolver.NXDOMAIN:
        return f"Domain '{domain}' does not exist."
    except dns.exception.Timeout:
        return "Timeout error during DNS query."
    except Exception as e:
        return f"Error: {e}"

# Example usage
domain_to_check = "http://www.ktplasmachinery.com/cs/"
dns_records = get_dns_records(domain_to_check)

print(f"DNS records for {domain_to_check}:\n{dns_records}")


In [ ]:
import requests
from bs4 import BeautifulSoup

def has_empty_title(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the title tag
            title_tag = soup.find('title')

            # Return 1 if the title is empty, otherwise return 0
            return 1 if title_tag is not None and not title_tag.text.strip() else 0
        
        else:
            return 0  # Return 0 for failure to retrieve data

    except Exception as e:
        return 0  # Return 0 for errors

# Example usage
website_url = "http://html.house/l7ceeid6.html"
result = has_empty_title(website_url)

print(f"The website {website_url} has an {'empty' if result else 'non-empty'} title.")


In [ ]:
import pandas as pd
import re

def create_url_features(url):
    features = {}
    
    # length_url
    features['length_url'] = len(url)
    
    # length_hostname
    features['length_hostname'] = len(url.split("/")[2])
    
    # nb_dot
    features['nb_dot'] = url.count(".")
    
    # nb_hyphens
    features['nb_hyphens'] = url.count("-")
    
    # nb_at
    features['nb_at'] = url.count("@")
    
    # nb_qm
    features['nb_qm'] = url.count("?")
    
    # nb_and
    features['nb_and'] = url.count("&")
    
    # nb_eq
    features['nb_eq'] = url.count("=")
    
    # nb_underscore
    features['nb_underscore'] = url.count("_")
    
    # nb_tilde
    features['nb_tilde'] = url.count("~")
    
    # nb_percentage
    features['nb_percentage'] = url.count("%")
    
    # nb_slash
    features['nb_slash'] = url.count("/")
    
    # nb_star
    features['nb_star'] = url.count("*")
    
    # nb_colon
    features['nb_colon'] = url.count(":")
    
    # nb_comma
    features['nb_comma'] = url.count(",")
    
    # nb_semicolumn
    features['nb_semicolumn'] = url.count(";")
    
    # nb_doller
    features['nb_doller'] = url.count("$")
    
    # nb_space
    features['nb_space'] = url.count(" ")
    
    # nb_www
    features['nb_www'] = url.count("www")
    
    # nb_com
    features['nb_com'] = url.count("com")
    
    # nb_dslash
    features['nb_dslash'] = url.count("//")
    
    # https token
    features['https_token'] = 0 if "https" in url else 1
    
    # http_in_path
    features['http_in_path'] = ("".join(url.split('/')[1:])).count("http")
    
    # shortening_service
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    features['shortening_service'] = 1 if match else 0
    
    return pd.DataFrame(features, index=[0])

# Example usage
url_to_check = "http://www.example.com"
url_features = create_url_features(url_to_check)

print(f"URL features for {url_to_check}:\n{url_features.columns}")



In [ ]:
import urllib.request
from bs4 import BeautifulSoup

def get_alexa_rank(url):
    try:
        # Try to fetch the Alexa rank data for the given URL
        rank = BeautifulSoup(urllib.request.urlopen(
            "http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
        
        # Convert the rank to an integer
        rank = int(rank)

        # Check if the rank is below 100,000
        if rank < 100000:
            # If the rank is below 100,000, return 1
            return 1
        else:
            # If the rank is 100,000 or above, return 0
            return 0
    except Exception as e:
        # If there is an exception (e.g., unable to fetch data), return -1
        return -1

# Example usage:
url_to_check = "example.com"
result = get_alexa_rank(url_to_check)

if result == 1:
    print(f"The website {url_to_check} has an Alexa rank below 100,000.")
elif result == 0:
    print(f"The website {url_to_check} has an Alexa rank 100,000 or above.")
else:
    print(f"Failed to retrieve Alexa rank for {url_to_check}.")


In [ ]:
df['page_rank'].unique()